### libraries

In [1]:
## remember to run conn_cocolab from the terminal before running cells in this notebook!
import os
import urllib, cStringIO
import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import datetime

# import svg_render_helpers as rdh
from scipy.spatial import distance

### file hierarchy and database connection vars

In [2]:
## directory & file hierarchy

iterationName = 'biganimalgame_run_v1'
analysis_dir = os.getcwd()


In [3]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://stanford:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
# coll = db['animalgame_run_v1']
coll = db['biganimalgame_run_v1']
this_collection = coll

### Preprocessing data 

Notes:
There are two types of records in the database: 'guessData' and 'surveyData'; guessData is the main data and surveyData is optional.
The session identifier is called "sessionId".




In [4]:
total_sessions = coll.find().distinct('sessionId') ## this returns ALL sessions in this collection.
print 'We currently have {} total sessions.'.format(len(total_sessions))

We currently have 1287 total sessions.


### Save out all data into variables that can be written into csv

In [5]:
coll.find().distinct('dataType')

[u'guessData']

In [6]:
### Open variables
clicked_category = []
intended_category = []
producer_age = []
recognizer_age = []
sessionId = []
sketch_path = []
RT = []
trial_num = []

In [7]:
interferenceCount = 0
for s in total_sessions:
    guesses = this_collection.find({'$and': [{'sessionId':s}, {'dataType':'guessData'}]}).sort('time')  
    this_survey = this_collection.find({'$and': [{'sessionId':s}, {'dataType':'survey'}]})
    if this_survey.count()>0:
        survey_answers = this_survey[0]
        interference = (survey_answers['other_guessed'] or survey_answers['parent_guessed'])
        if interference == True:
            print('found interference...')
            interferenceCount = interferenceCount+1
    else:
        interference = False
    if (guesses.count()>5 & interference==False): ## did we get past practice trials and have no interference?
        for guess in guesses:        
            clicked_category.append(guess['clicked_category'])
            intended_category.append(guess['intended_category'])
            producer_age.append(guess['producer_age'])
            recognizer_age.append(guess['recognizer_age'])
#             print('age = ' +  guess['recognizer_age'])
            sessionId.append(guess['sessionId'])
            sketch_path.append(guess['sketch_path'])
            trial_num.append(guess['trialNum'])

            this_RT = guess['endTrialTime'] - guess['startTrialTime']
            RT.append(this_RT)

In [8]:
recognizer_age

[u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'adult',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age7',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age6',
 u'age3',
 u'age3'

### Save out data into a dataframe

In [9]:
## compile data
really_run_this = 1 
if really_run_this:
    X = pd.DataFrame([sessionId, trial_num, recognizer_age, sketch_path, intended_category, producer_age, clicked_category, RT])
    X = X.transpose()
    X.columns = ['sessionId', 'trial_num', 'recognizer_age', 'sketch_path', 'intended_category', 'producer_age', 'clicked_category', 'RT']
    X.to_csv('biganimalgame_out_final_april2020.csv')             